# Implementación Módulo 2

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("processed_df.csv", sep=";", low_memory=False)

Vemos que algunos posts al hacer el preprocesamiento como tenían solo caracteres no alfanumericos ahora contienen Nan.

Hay que eliminarlos

In [6]:
df.isna().sum()

created_date       0
subreddit          0
title              0
author             0
full_link          0
score              0
post            1962
sentiment          0
dtype: int64

In [7]:
df[df.post.isna()]

,created_date,subreddit,title,author,full_link,score,post,sentiment
446,2011-02-11 08:04:10,statistics,weighting with r,[deleted],https://www.reddit.com/r/statistics/comments/f...,1,NaN,1
605,2011-04-11 17:54:51,datasets,Where can i download netflix database for my e...,Exibus,https://www.reddit.com/r/datasets/comments/gnf...,1,NaN,1
1631,2011-12-04 19:38:20,AskStatistics,"If correlation is not necessarily causation, h...",JimJamieJames,https://www.reddit.com/r/AskStatistics/comment...,0,NaN,1
1698,2011-12-14 18:29:01,statistics,If you have a series of points that are better...,[deleted],https://www.reddit.com/r/statistics/comments/n...,1,NaN,1
2400,2012-04-02 20:39:37,datasets,Archives bends under rush for 1940 census reco...,berlinbrown,https://www.reddit.com/r/datasets/comments/rps...,0,NaN,0
...,...,...,...,...,...,...,...,...
272822,2022-05-01 01:01:57,artificial,"Sentient microwave develops PTSD, attempts to ...",Nekrofeeelyah,https://www.reddit.com/r/artificial/comments/u...,1,NaN,0
273079,2022-05-02 13:47:19,datascience,Deep Neural Networks to Detect Weeds from Crop...,SpecificAd3444,https://www.reddit.com/r/datascience/comments/...,1,NaN,0
273477,2022-05-04 15:34:14,dataengineering,Dockerizing an Apache Spark Standalone Cluster,ramses-coraspe,https://www.reddit.com/r/dataengineering/comme...,1,NaN,0
273531,2022-05-04 20:01:57,dataengineering,Build an ETL pipeline with Apache Airflow and ...,ramses-coraspe,https://www.reddit.com/r/dataengineering/comme...,1,NaN,0


In [8]:
df.drop(df[df.post.isna()].index, inplace=True)

# TFIDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression

In [10]:
tfidf = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf.fit_transform(df.post)

In [11]:
df.subreddit.unique()

array(['artificial', 'statistics', 'MachineLearning', 'computervision',
       'rstats', 'analytics', 'datasets', 'computerscience',
       'AskStatistics', 'data', 'datascience', 'MLQuestions',
       'DataScienceJobs', 'deeplearning', 'dataengineering',
       'dataanalysis', 'learnmachinelearning', 'kaggle',
       'datascienceproject'], dtype=object)

In [12]:
encoder = LabelEncoder()
Y = encoder.fit_transform(df.subreddit)

In [13]:
X_tfidf

<272241x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 7941020 stored elements in Compressed Sparse Row format>

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, Y, test_size=0.3, random_state=42)

In [15]:
X_train

<190568x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 5562764 stored elements in Compressed Sparse Row format>

In [16]:
Y_train

array([ 3,  7, 10, ...,  6,  0, 18], shape=(190568,))

In [17]:
from tqdm import tqdm


In [18]:
"""random_forest = RandomForestClassifier(n_jobs=-1, random_state=42)
random_forest.fit(X_train, Y_train)
#15 min """

'random_forest = RandomForestClassifier(n_jobs=-1, random_state=42)\nrandom_forest.fit(X_train, Y_train)\n#15 min '

In [19]:
"""logreg = LogisticRegression(max_iter=1000, n_jobs=-1)
logreg.fit(X_train, Y_train)
#1 min"""

'logreg = LogisticRegression(max_iter=1000, n_jobs=-1)\nlogreg.fit(X_train, Y_train)\n#1 min'

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
#from xgboost import XGBClassifier
import time

# Datos de entrada y etiquetas
X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, Y, test_size=0.2, random_state=42)

# Lista de modelos
models = {
    #"Logistic Regression": LogisticRegression(max_iter=1000, n_jobs=-1),
    #"Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    #"XGBoost": XGBClassifier(n_estimators=100, tree_method='gpu_hist', max_depth=3, learning_rate=0.1, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(max_depth=3,random_state=42),
    "Naive Bayes": MultinomialNB()
}

# Probar cada modelo
results = {}
for model_name, model in models.items():
    print(f"Entrenando: {model_name}")
    start = time.time()
    model.fit(X_train, Y_train)  # Entrenar modelo
    end = time.time()
    Y_pred = model.predict(X_test)  # Predecir etiquetas
    
    # Evaluar el modelo
    accuracy = accuracy_score(Y_test, Y_pred)
    f1score = f1_score(Y_test, Y_pred, average='weighted')
    print(f"Tiempo de entrenamiento de {model_name}: {end - start:.2f}s")
    print(f"{model_name} Accuracy: {accuracy:.2f}")
    print(f"{model_name} F1 Score: {f1score:.2f}")
    print(classification_report(Y_test, Y_pred, target_names=encoder.classes_))
    # Guardar resultados
    results[model_name] = accuracy

# Mostrar el mejor modelo
best_model = max(results, key=results.get)
print(f"El mejor modelo es: {best_model} con una precisión de {results[best_model]:.2f}")


Entrenando: Gradient Boosting
Tiempo de entrenamiento de Gradient Boosting: 31007.85s
Gradient Boosting Accuracy: 0.46
Gradient Boosting F1 Score: 0.45
                      precision    recall  f1-score   support

       AskStatistics       0.47      0.51      0.49      5989
     DataScienceJobs       0.83      0.55      0.66       458
         MLQuestions       0.21      0.03      0.05      2254
     MachineLearning       0.34      0.55      0.42      7365
           analytics       0.66      0.55      0.60      1611
          artificial       0.53      0.45      0.49      1721
     computerscience       0.57      0.65      0.60      4511
      computervision       0.49      0.49      0.49      1853
                data       0.57      0.20      0.30       544
        dataanalysis       0.33      0.15      0.20       825
     dataengineering       0.66      0.53      0.59      1648
         datascience       0.51      0.59      0.55      7632
  datascienceproject       0.00      0.00

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [21]:
# 7. Evaluamos los modelos
Y_pred2 = logreg.predict(X_test)
#Y_pred = model.predict(X_test)
#print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Accuracy:", accuracy_score(Y_test, Y_pred2))
print("Classification Report:\n", classification_report(Y_test, Y_pred2))
#print("Classification Report:\n", classification_report(Y_test, Y_pred))

NameError: name 'logreg' is not defined

In [ ]:
from sklearn.metrics import f1_score

Y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Classification Report:\n", f1_score(Y_test, Y_pred, average='weighted'))

Accuracy: 0.4653528990431413
Classification Report:
 0.44672030785914674


In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp(df.post[0])
for entity in doc.ents:
    print(entity.text, entity.label_)